In [11]:
import numpy as np # Importing NumPy for high-performance vector calculations.

class WaterTwinCore:
    """
    WaterTwin AI - Digital Twin Simulation Engine
    This class serves as the mathematical core for simulating water harvesting systems.
    """
    def __init__(self, roof_area, tank_capacity, runoff_coeff=0.9):
        # Parameters are set based on the standards defined in the project README.
        self.roof_area = roof_area          # Measured in square meters (m2).
        self.tank_capacity = tank_capacity  # Measured in Liters (L).
        self.runoff_coeff = runoff_coeff    # Runoff Coefficient (Default: 0.9 represents 90% efficiency).
        
        # Instantaneous state of the system
        self.current_water_liters = 0 

    def predict_harvest(self, rain_mm):
        """
        Mathematical Model: Volume (L) = Rainfall (mm) x Area (m2) x Runoff Coeff.
        This function converts linear rainfall depth into volume.
        """
        # We utilize NumPy's multiply function to establish a foundation for vector-based 
        # calculations, allowing the engine to process arrays of weather data in the future.
        harvest = np.multiply(rain_mm, self.roof_area * self.runoff_coeff)
        return np.round(harvest, 2)

    def run_simulation(self, rain_prediction_mm, initial_fill_percent, daily_usage_l):
        """
        Scenario Analysis & Overflow Forecasting
        Simulates how the system will react to predicted rainfall and planned consumption.
        """
        # Step 1: Calculate the starting water volume based on the initial fill percentage.
        start_water = (initial_fill_percent / 100) * self.tank_capacity
        
        # Step 2: Determine the expected water inflow from the roof.
        inflow = self.predict_harvest(rain_prediction_mm)
        
        # Step 3: Calculate the potential total water (Current + Inflow - Consumption).
        potential_total = start_water + inflow - daily_usage_l
        
        # Step 4: Overflow Forecasting.
        # np.maximum(0, ...) ensures we don't get negative overflow values.
        overflow = np.maximum(0, potential_total - self.tank_capacity)
        
        # Step 5: Final tank level calculation.
        # np.clip(..., 0, capacity) ensures the level stays within physical boundaries (0 to max capacity).
        final_level = np.clip(potential_total, 0, self.tank_capacity)
        fill_rate = (final_level / self.tank_capacity) * 100
        
        # Returning a structured dictionary for the UI or API consumption.
        return {
            "predicted_inflow_L": float(inflow),
            "predicted_overflow_L": float(overflow),
            "final_fill_rate": round(float(fill_rate), 1),
            "status": "OVERFLOW RISK" if overflow > 0 else "STABLE"
        }

print("Digital Twin Simulation is ready to use.")

Digital Twin Simulation is ready to use.


In [12]:
import joblib

# README Example: Configuration with 500m2 roof area and 15,000L tank capacity
watertwin_v1 = WaterTwinCore(roof_area=500, tank_capacity=15000)

# Serializing (freezing) and saving the engine instance to the local file system for future use
joblib.dump(watertwin_v1, 'models/watertwin_engine.joblib')

print("The simulation engine was frozen and saved to the 'models/' folder.")

The simulation engine was frozen and saved to the 'models/' folder.


In [ ]:
%%writefile app.py
import streamlit as st
import joblib
import os
from dotenv import load_dotenv # Added to load the .env file

# 1. LOAD ENVIRONMENT VARIABLES
load_dotenv() # This looks for a .env file and loads the variables

# Retrieve the key from .env (Make sure the name matches exactly what is in your .env file)
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY") 

try:
    from google import genai
except ImportError:
    st.error("The 'google-genai' library is not installed. Please run 'pip install google-genai'.")

# 2. PAGE CONFIGURATION
st.set_page_config(page_title="WaterTwin AI", page_icon=None, layout="wide")

# 3. API CLIENT INITIALIZATION
if not GEMINI_API_KEY:
    st.error("API Key not found! Please check your .env file for GOOGLE_API_KEY.")
else:
    try:
        client = genai.Client(api_key=GEMINI_API_KEY)
    except Exception as e:
        st.error(f"API Connection Error: {e}")

@st.cache_resource
def load_engine():
    if os.path.exists('models/watertwin_engine.joblib'):
        return joblib.load('models/watertwin_engine.joblib')
    else:
        st.error("Model file not found.")
        return None

engine = load_engine()

if engine and GEMINI_API_KEY:
    # 4. SIDEBAR - USER INPUTS
    with st.sidebar:
        st.header("System Configurations")
        engine.roof_area = st.number_input("Roof Area (m2)", value=500)
        engine.tank_capacity = st.number_input("Tank Capacity (L)", value=15000)
        
        st.divider()
        st.header("Weather Forecast")
        rain_val = st.slider("Expected Rainfall (mm)", 0.0, 100.0, 25.0)
        usage_val = st.number_input("Estimated Consumption (L)", value=300)
        current_fill = st.slider("Current Fill Level (%)", 0, 100, 40)

    # 5. SIMULATION CALCULATION
    results = engine.run_simulation(rain_val, current_fill, usage_val)

    # 6. DASHBOARD VISUALIZATION
    st.title("WaterTwin AI Dashboard")
    col1, col2, col3 = st.columns(3)

    with col1:
        st.metric("Inflow Water", f"{results['predicted_inflow_L']} L")
    with col2:
        st.metric("Overflow Risk", f"{results['predicted_overflow_L']} L", delta_color="inverse")
    with col3:
        st.metric("Predicted Fill Level", f"%{results['final_fill_rate']}")

    # 7. AI ADVISOR LAYER
    st.divider()
    st.subheader("AI Strategy Report")

    if st.button("Start AI Analysis"):
        with st.spinner("Gemini is interpreting data..."):
            prompt_text = f"""
            You are a water management expert. Interpret these results:
            - Fill Level: {results['final_fill_rate']}%
            - Inflow: {results['predicted_inflow_L']}L
            - Overflow: {results['predicted_overflow_L']}L
            Provide 3 sentences of action-oriented advice.
            """
            response = client.models.generate_content(
                model="gemini-2.0-flash",
                contents=prompt_text
            )
            st.write(response.text)

Overwriting app.py
